# Optimization techniques Lab. 4: (Reduced) Variable Neighborhood Search
## Introduction
**Goal.** The goal of this lab is to compare the behavior of VNS and RVNS on the knapsack 0/1 problem.

**Getting started.** The following cells contain the implementation of the methods that we will use throughout this lab, together with utilities.

## The Knapsack 0/1 problem
The knapsack 0/1 problem is a combinatorial problem that works as follows.

We have a bag with limited capacity L, and we have a set of items $I = {i_1, ..., i_N}$
Each $i_j$ has a given volume $w_{i_j}$ and a value $v_{i_j}$.
The goal of the optimization problem is to fill the bag with a combination of items $S$ such that the combination maximizes the total value contained by the bag while complaining with the constraint on the capacity.

In this version of the knapsack problem we can either discard or carry (at most 1) item. Thus, the search space is $\{0, 1\}^{|I|}$.

Since this is a maximization problem, we can turn it into a minimization problem by returning the opposite of the value.
If a solution is not feasible (i.e., exceeds the maximum volume) the function will return a score of 0 (like an empty bag).

In [ ]:
import random
import itertools as it

from matplotlib import pyplot as plt
from typing import Callable, List, Tuple
from collections import Counter

In [ ]:
class Knapsack_0_1:

    def __init__(self):
        self._items = [
            {'name': 'apricot', 'value': 1, 'volume': 1},
            {'name': 'apple', 'value': 1, 'volume': 2},

            {'name': 'cherry', 'value': 2, 'volume': 1},
            {'name': 'pear', 'value': 2, 'volume': 2},
            {'name': 'banana', 'value': 2, 'volume': 2},

            {'name': 'blueberry', 'value': 3, 'volume': 1},
            {'name': 'orange', 'value': 3, 'volume': 2},
            {'name': 'avocado', 'value': 3, 'volume': 2},

            {'name': 'coconut', 'value': 4, 'volume': 3},

            {'name': 'watermelon', 'value': 5, 'volume': 10},
        ]
        self._BAG_CAPACITY = 10
        self.history = []
        self.values = []

    def _get_value(self, solution):
        cur_cap = self._BAG_CAPACITY
        cur_val = 0
        for i, v in enumerate(solution):
            if v == 1:
                cur_val += self._items[i]['value']
                cur_cap -= self._items[i]['volume']
            if cur_cap < 0:
                return 0
        return -cur_val

    def __call__(self, solution):
        value = self._get_value(solution)
        self.history.append(solution)
        self.values.append(value)
        return value

    def trend(self, path):
        # 4 + 3 + 2 + 1 + 3 + 3 = 16
        # 3 + 1 + 1 + 1 + 2 + 2 = 10
        plt.figure()
        plt.plot(self.values)
        plt.axhline(-16, color="r", label="optimum")
        if path is None:
            plt.show()
        else:
            plt.savefig(path, dpi=400)
        plt.close()

In [ ]:
def shake(
        x: List[bool],
        k: int,
        bits: int
) -> List[bool]:
    ss = Counter(random.choices(range(len(x)), k=bits * k))
    return [e if ss.get(i, 0) % 2 == 0 else not e for i, e in enumerate(x)]


def move_or_not(
        fn: Callable[[List[bool]], int],
        x: List[bool],
        xp: List[bool],
        k: int
) -> Tuple[List[bool], int]:
    return (xp, 1) if fn(xp) < fn(x) else (x, k + 1)


def local_search_best(
        fn: Callable[[List[bool]], int],
        x: List[bool],
        bits: int
) -> List[bool]:
    xps = (
        [e if i in s else not e for (i, e) in enumerate(x)]
        for s
        in it.chain.from_iterable(it.combinations(x, r) for r in range(bits + 1))
    )
    return min(xps, key=fn)


def local_search_first(
        fn: Callable[[List[bool]], int],
        x: List[bool],
        bits: int
) -> List[bool]:
    xps = (
        [e if i in s else not e for (i, e) in enumerate(x)]
        for s
        in it.chain.from_iterable(it.combinations(x, r + 1) for r in range(bits))
    )
    fnx = fn(x)
    return next(filter(lambda xp: fn(xp) < fnx, xps), x)


def vns(
        fn: Callable[[List[bool]], int],
        x0: List[bool],
        kmax: int,
        local_search: Callable[[Callable[[List[bool]], int], List[bool], int], List[bool]],
        shake_bits: int,
        search_bits: int,
) -> None:
    """
    Seeks for the minimum of the function by means of the Variable Neighborhood
    Search algorithm.

    :f: the function to optimize
    :x0: the initial point
    :kmax: the max number of iterations for each neighborhood
    """
    x = [e for e in x0]
    k = 0
    while k < kmax:
        x, k = move_or_not(fn, x, local_search(fn, shake(x, k, shake_bits), search_bits), k)


def rvns(
        fn: Callable[[List[bool]], int],
        x0: List[bool],
        kmax: int,
        shake_bits: int,
) -> None:
    """
    Seeks for the minimum of the function by means of the Reduced Variable
    Neighborhood Search algorithm.

    :f: the function to optimize
    :x0: the initial point
    :kmax: the max number of iterations for each neighborhood
    """
    x = [e for e in x0]
    k = 0
    while k < kmax:
        x, k = move_or_not(fn, x, shake(x, k, shake_bits), k)

# Variable Neighborhood Search
---
## Questions:
- how does the starting point influence the search process?
- how does the kmax parameter affect the quality of the result?
- how does the generation of the neighborhood affect:
    - quality of the search?
    - velocity of the search?

In [ ]:
x0s = [[False] * 10, [i % 2 != 0 for i in range(10)]]

for x0 in x0s + [[not e for e in x0] for x0 in x0s]:
    for kmax in [75]:
        for shake_bits in [1, 2, 3]:
            for search_bits in range(1, shake_bits):
                for name, local_search in [('first', local_search_first), ('best', local_search_best)]:
                    random.seed(7)
                    func = Knapsack_0_1()
                    vns(
                        fn=func,
                        x0=x0,
                        kmax=kmax,
                        local_search=local_search,
                        shake_bits=shake_bits,
                        search_bits=search_bits
                    )
                    func.trend(f'out/{name} x0 : {x0} kmax : {kmax} shake : {shake_bits} search : {search_bits}.svg')

# Reduced Variable Neighborhood Search
---
## Questions:
- how does the starting point influence the search process?
- how does the kmax parameter affect the quality of the result?
- how does the generation of the neighborhood affect:
    - quality of the search?
    - velocity of the search?
- how does RVNS compare to VNS?

In [ ]:
x0s = [[False] * 10, [i % 2 != 0 for i in range(10)]]

for x0 in x0s + [[not e for e in x0] for x0 in x0s]:
    for kmax in map(lambda i: 10 * i, [75]):
        for shake_bits in [1, 2, 3]:
            random.seed(7)
            func = Knapsack_0_1()
            rvns(
                fn=func,
                x0=x0,
                kmax=kmax,
                shake_bits=shake_bits
            )
            func.trend(f'out/x0 : {x0} kmax : {kmax} shake : {shake_bits}.svg')